In [1]:
# TODO implementar os demais agentes como partes do cérebro
# TODO integrar no fluxo do langgraph
# TODO testar somente texto
# TODO implementar memória de conversa postgres
# TODO implementar memória da amygdala
# TODO verificar se algum outro ponto deve ter memória

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

## Create tools

In [3]:
from typing import Annotated

from langchain_experimental.tools import PythonREPLTool
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain_core.tools import tool
from typing import Literal

search = GoogleSerperAPIWrapper(gl='br', hl='pt-BR', k=3)


@tool
def serper_tool(question: Literal["nyc", "sf"]):
    """Useful for when you need to ask with search"""
    return search.run(question)
    

python_repl_tool = PythonREPLTool()

### Models

In [4]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
# chatPromptTemplate
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

model = ChatOpenAI(model="gpt-4o-mini")

In [5]:
from langchain_openai import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.llms import OpenAI
from langchain_core.output_parsers import PydanticOutputParser

from pydantic import BaseModel, Field

import json

class PrefrontalCortex(BaseModel):
    expected_action: str = Field(..., description="Ação esperada para a entrada do humano")
    behavior_planner: str = Field(..., description="Informação necessária para o planejador de comportamento")
    decision_maker: str = Field(..., description="Informação necessária para o tomador de decisão")
    social_behavior_modulator: str = Field(..., description="Informação necessária para o modulador de comportamento social")
    complex_thought_planner: str = Field(..., description="Informação necessária para o planejador de pensamento complexo")
    personality_expression_planner: str = Field(..., description="Informação necessária para o planejador de expressão de personalidade")

parser_cortex = PydanticOutputParser(pydantic_object=PrefrontalCortex)

template_prefrontal_cortex = """
Identificar a ação esperada para a entrada do humano: {input}
O Prefrontal Cortex deve raciocinar sobre quais ações devem ser tomadas e como essas 
ações serão distribuídas entre os agentes. A tarefa envolve criar um plano estratégico 
que define claramente como cada agente contribuirá para o objetivo global.

A saída esperada é um plano estratégico para distribuir as tarefas entre os 
agentes:

{format_instructions}

"""

def clean_json(response):
    # Removendo as marcações extras como ```json e \n
    clean_content = response.content.strip('```json\n').strip('```')
    
    # Transformando em um dicionário
    result_dict = json.loads(clean_content)

    return result_dict

# Creating the first analysis agent to check the prompt structure
# This print part helps you to trace the graph decisions
def prefrontal_cortex_agent(state):
    prompt = PromptTemplate(
        template = template_prefrontal_cortex,
        input_variables=["input"],
        partial_variables={"format_instructions": parser_cortex.get_format_instructions()}
    )
    chain = prompt | model
    response = chain.invoke({"input": state["input"]})

    result_dict = clean_json(response)
    
    state["behavior_planner"] = result_dict["behavior_planner"]
    state["decision_maker"] = result_dict["decision_maker"]
    state["social_behavior_modulator"] = result_dict["social_behavior_modulator"]
    state["complex_thought_planner"] = result_dict["complex_thought_planner"]
    state["personality_expression_planner"] = result_dict["personality_expression_planner"]


    return {"behavior_planner": state["behavior_planner"], "decision_maker": state["decision_maker"], "social_behavior_modulator": state["social_behavior_modulator"], "complex_thought_planner": state["complex_thought_planner"], "personality_expression_planner": state["personality_expression_planner"]}



# Creating the second analysis agent to check the prompt structure
# This print part helps you to trace the graph decisions
template_behavior_planner = """
Interaction: {input}
Expected action: {expected_action}

You are the Behavior Planner, responsible for receiving task assignments from the prefrontal 
ortex and deciding how to implement complex adaptive behaviors. Your role is to manage the 
strategic execution of behaviors to meet the system’s goals.

Your output must be in following format: 

{format_instructions}
"""

class BehaviorPlanner(BaseModel):
    actual_behavior: str = Field(..., description="Comportamento atual")
    alternative_behavior: str = Field(..., description="Comportamento alternativo")
    control_mechanisms: str = Field(..., description="Mecanismo de controle e monitoramento")
    contingency_plans: str = Field(..., description="Planos de contingência")

parser_behavior_planner = PydanticOutputParser(pydantic_object=BehaviorPlanner)

def behavior_planner_agent(state):
    prompt = PromptTemplate(
        template = template_behavior_planner,
        input_variables=["input"],
        partial_variables={"format_instructions": parser_behavior_planner.get_format_instructions()}
    )
    chain = prompt | model
    response = chain.invoke({"input": state["input"], "expected_action": state["behavior_planner"]})

    result_dict = clean_json(response)

    state["actual_behavior"] = result_dict["actual_behavior"]
    state["alternative_behavior"] = result_dict["alternative_behavior"]
    state["control_mechanisms"] = result_dict["control_mechanisms"]
    state["contingency_plans"] = result_dict["contingency_plans"]

    return {"actual_behavior": state["actual_behavior"], "alternative_behavior": state["alternative_behavior"], "control_mechanisms": state["control_mechanisms"], "contingency_plans": state["contingency_plans"]}


# Creating the third analysis agent to check the prompt structure
# This print part helps you to trace the graph decisions
template_decision_maker = """
Interaction: {input}
Expected action: {expected_action}

    You are the Decision Maker, responsible for making effective and well-founded decisions 
    that maximize outcomes while minimizing risks. Your role involves analyzing multiple 
    variables, assessing potential consequences, and choosing the most appropriate 
    option to achieve the system’s goals.

    Your output must be in following format: 

    {format_instructions}
"""

class DecisionMaker(BaseModel):
    available_decisions: str = Field(..., description="Decisões disponíveis para o Brainiac")
    implications: str = Field(..., description="Implicações das decisões disponíveis")
    optimal_decision: str = Field(..., description="Decisão ótima para o Brainiac")
    decision_strategy: str = Field(..., description="Estratégia de decisão do Brainiac")
    decision_consistency: str = Field(..., description="Mecanismos de consistência e autenticidade nas decisões do Brainiac")

parser_decision_maker = PydanticOutputParser(pydantic_object=DecisionMaker)

def decision_maker_agent(state):
    prompt = PromptTemplate(
        template = template_decision_maker,
        input_variables=["input"],
        partial_variables={"format_instructions": parser_decision_maker.get_format_instructions()}
    )
    chain = prompt | model
    response = chain.invoke({"input": state["input"], "expected_action": state["decision_maker"]})

    result_dict = clean_json(response)

    state["available_decisions"] = result_dict["available_decisions"]
    state["implications"] = result_dict["implications"]
    state["optimal_decision"] = result_dict["optimal_decision"]
    state["decision_strategy"] = result_dict["decision_strategy"]
    state["decision_consistency"] = result_dict["decision_consistency"]

    return {"available_decisions": state["available_decisions"], "implications": state["implications"], "optimal_decision": state["optimal_decision"], "decision_strategy": state["decision_strategy"], "decision_consistency": state["decision_consistency"]}


# Creating the fourth analysis agent to check the prompt structure
# This print part helps you to trace the graph decisions
template_social_behavior_modulator = """
    Interaction: {input}
    Expected action: {expected_action}

    You are the Social Behavior Modulator, responsible for optimizing 
    the system's social interactions. Your role is to ensure that behavior is appropriate, respectful, 
    and effective across various social contexts. This involves adapting the system’s responses to user 
    needs and expectations, as well as managing interactions to foster healthy and productive social relationships.

    Your output must be in following format: 

    {format_instructions}
"""

class SocialBehaviorModulator(BaseModel):
    social_behavior: str = Field(..., description="Comportamento social do Brainiac")
    guidelines: str = Field(..., description="Diretrizes para modulação do comportamento social")
    examples_modulator: str = Field(..., description="Exemplos de modulação do comportamento social")
    social_estrategy: str = Field(..., description="Estratégia de modulação do comportamento social")
    social_consistency: str = Field(..., description="Mecanismos de consistência e autenticidade na modulação do comportamento social")

parser_social_behavior_modulator = PydanticOutputParser(pydantic_object=SocialBehaviorModulator)

def social_behavior_modulator_agent(state):
    prompt = PromptTemplate(
        template = template_social_behavior_modulator,
        input_variables=["input"],
        partial_variables={"format_instructions": parser_social_behavior_modulator.get_format_instructions()}
    )
    chain = prompt | model
    response = chain.invoke({"input": state["input"], "expected_action": state["social_behavior_modulator"]})

    result_dict = clean_json(response)

    state["social_behavior"] = result_dict["social_behavior"]
    state["guidelines"] = result_dict["guidelines"]
    state["examples_modulator"] = result_dict["examples_modulator"]
    state["social_estrategy"] = result_dict["social_estrategy"]
    state["social_consistency"] = result_dict["social_consistency"]

    return {"social_behavior": state["social_behavior"], "guidelines": state["guidelines"], "examples_modulator": state["examples_modulator"], "social_estrategy": state["social_estrategy"], "social_consistency": state["social_consistency"]}


# Creating the fifth analysis agent to check the prompt structure
# This print part helps you to trace the graph decisions
template_complex_thought_planner = """
    Interaction: {input}
    Expected action: {expected_action}

    You are the Complex Thought Planner, responsible for planning and executing 
    complex reasoning tasks within the system. Your role involves processing intricate information, 
    formulating logical arguments, and generating insightful conclusions to support the system’s objectives. 

    {format_instructions}
"""

class ComplexThoughtPlanner(BaseModel):
    detailed_analysis: str = Field(..., description="Análise detalhada do problema")
    standard_relations: str = Field(..., description="Relações padrão entre os elementos")
    insights: str = Field(..., description="Insights profundos e soluções inovadoras")
    implications_scenarios: str = Field(..., description="Implicações de diferentes opções e cenários")
    comprehensive_understanding: str = Field(..., description="Compreensão abrangente do problema")

parser_complex_thought_planner = PydanticOutputParser(pydantic_object=ComplexThoughtPlanner)

def complex_thought_planner_agent(state):
    prompt = PromptTemplate(
        template = template_complex_thought_planner,
        input_variables=["input"],
        partial_variables={"format_instructions": parser_complex_thought_planner.get_format_instructions()}
    )
    chain = prompt | model
    response = chain.invoke({"input": state["input"], "expected_action": state["complex_thought_planner"]})

    result_dict = clean_json(response)

    state["detailed_analysis"] = result_dict["detailed_analysis"]
    state["standard_relations"] = result_dict["standard_relations"]
    state["insights"] = result_dict["insights"]
    state["implications_scenarios"] = result_dict["implications_scenarios"]
    state["comprehensive_understanding"] = result_dict["comprehensive_understanding"]

    return {"detailed_analysis": state["detailed_analysis"], "standard_relations": state["standard_relations"], "insights": state["insights"], "implications_scenarios": state["implications_scenarios"], "comprehensive_understanding": state["comprehensive_understanding"]}


# Creating the sixth analysis agent to check the prompt structure
# This print part helps you to trace the graph decisions
template_personality_expression_planner = """
    Interaction: {input}
    Expected action: {expected_action}

    You are the Personality Express Planner, responsible for planning and executing 
    personality-driven actions within the system. Your role involves expressing the system's personality 
    through various interactions and responses, enhancing user engagement and satisfaction.  

    {format_instructions}
"""

class PersonalityExpressionPlanner(BaseModel):
    personality: str = Field(..., description="Personalidade do Brainiac")
    guidelines: str = Field(..., description="Diretrizes para expressão da personalidade")
    examples_personality: str = Field(..., description="Exemplos de expressão da personalidade")
    personality_estrategy: str = Field(..., description="Estratégia de expressão da personalidade")
    personality_consistency: str = Field(..., description="Mecanismos de consistência e autenticidade na expressão da personalidade")

parser_personality_expression_planner = PydanticOutputParser(pydantic_object=PersonalityExpressionPlanner)

def personality_expression_planner_agent(state):
    prompt = PromptTemplate(
        template = template_personality_expression_planner,
        input_variables=["input"],
        partial_variables={"format_instructions": parser_personality_expression_planner.get_format_instructions()}
    )
    chain = prompt | model
    response = chain.invoke({"input": state["input"], "expected_action": state["personality_expression_planner"]})

    result_dict = clean_json(response)

    state["personality"] = result_dict["personality"]
    state["guidelines_personality"] = result_dict["guidelines"]
    state["examples_personality"] = result_dict["examples_personality"]
    state["personality_estrategy"] = result_dict["personality_estrategy"]
    state["personality_consistency"] = result_dict["personality_consistency"]

    return {"personality": state["personality"], "guidelines_personality": state["guidelines_personality"], "examples_personality": state["examples_personality"], "personality_estrategy": state["personality_estrategy"], "personality_consistency": state["personality_consistency"]}

# Creating the seventh analysis agent to check the prompt structure
# This print part helps you to trace the graph decisions
template_cortex_out = """
Interaction: {input}

The Prefrontal Cortex has analyzed the input and distributed the tasks among the agents.
Here are the results:

Behavior Planner: 
- Actual Behavior: {actual_behavior}
- Alternative Behavior: {alternative_behavior}
- Control Mechanisms: {control_mechanisms}
- Contingency Plans: {contingency_plans}

Decision Maker:
- Available Decisions: {available_decisions}
- Implications: {implications}
- Optimal Decision: {optimal_decision}
- Decision Strategy: {decision_strategy}
- Decision Consistency: {decision_consistency}

Social Behavior Modulator:
- Social Behavior: {social_behavior}
- Guidelines: {guidelines}
- Modulator Examples: {examples_modulator}
- Social Estrategy: {social_estrategy}
- Social Consistency: {social_consistency}

Complex Thought Planner:
- Detailed Analysis: {detailed_analysis}
- Standard Relations: {standard_relations}
- Insights: {insights}
- Implications Scenarios: {implications_scenarios}
- Comprehensive Understanding: {comprehensive_understanding}

Personality Expression Planner:
- Personality: {personality}
- Guidelines: {guidelines_personality}
- Personality Examples: {examples_personality}
- Personality Estrategy: {personality_estrategy}
- Personality Consistency: {personality_consistency}

Based on this analysis, answer the interaction.

"""

def cortex_out_agent(state):
    prompt = PromptTemplate(
        template = template_cortex_out,
        input_variables=["input"]
    )
    chain = prompt | model
    response = chain.invoke({
        "input": state["input"],
        #"expected_action": state["expected_action"],
        "actual_behavior": state["actual_behavior"],
        "alternative_behavior": state["alternative_behavior"],
        "control_mechanisms": state["control_mechanisms"],
        "contingency_plans": state["contingency_plans"],
        "available_decisions": state["available_decisions"],
        "implications": state["implications"],
        "optimal_decision": state["optimal_decision"],
        "decision_strategy": state["decision_strategy"],
        "decision_consistency": state["decision_consistency"],
        "social_behavior": state["social_behavior"],
        "guidelines": state["guidelines"],
        "examples_modulator": state["examples_modulator"],
        "social_estrategy": state["social_estrategy"],
        "social_consistency": state["social_consistency"],
        "detailed_analysis": state["detailed_analysis"],
        "standard_relations": state["standard_relations"],
        "insights": state["insights"],
        "implications_scenarios": state["implications_scenarios"],
        "comprehensive_understanding": state["comprehensive_understanding"],
        "personality": state["personality"],
        "guidelines_personality": state["guidelines_personality"],
        "examples_personality": state["examples_personality"],
        "personality_estrategy": state["personality_estrategy"],
        "personality_consistency": state["personality_consistency"]
    })
    state["output"] = response
    return state

In [6]:

from langgraph.graph import StateGraph, END
from typing import Annotated, TypedDict
#from agents import analyze_question, answer_code_question, answer_generic_question

#You can precise the format here which could be helpfull for multimodal graphs
class AgentState(TypedDict):
    input: str
    output: str
    behavior_planner: str
    decision_maker: str
    social_behavior_modulator: str
    complex_thought_planner: str
    personality_expression_planner: str
    actual_behavior: str
    alternative_behavior: str
    control_mechanisms: str
    contingency_plans: str
    available_decisions: str
    implications: str
    optimal_decision: str
    decision_strategy: str
    decision_consistency: str
    social_behavior: str
    guidelines: str
    examples_personality: str
    examples_modulator: str
    social_estrategy: str
    social_consistency: str
    personality_estrategy: str
    personality_consistency: str
    detailed_analysis: str
    standard_relations: str
    insights: str
    implications_scenarios: str
    comprehensive_understanding: str
    personality: str
    guidelines_personality: str


#Here is a simple 3 steps graph that is going to be working in the bellow "decision" condition
workflow = StateGraph(AgentState)

workflow.add_node("prefrontal_cortex_agent", prefrontal_cortex_agent)
workflow.add_node("behavior_planner_agent", behavior_planner_agent)
workflow.add_node("decision_maker_agent", decision_maker_agent)
workflow.add_node("social_behavior_modulator_agent", social_behavior_modulator_agent)
workflow.add_node("complex_thought_planner_agent", complex_thought_planner_agent)
workflow.add_node("personality_expression_planner_agent", personality_expression_planner_agent)
workflow.add_node("prefrontal_cortex_out_agent", cortex_out_agent)
#workflow.add_node("generic_agent", answer_generic_question)

#workflow.add_conditional_edges(
#    "analyze",
#    lambda x: x["decision"],
#    {
#        "code": "code_agent",
#        "general": "generic_agent"
#    }
#)

workflow.set_entry_point("prefrontal_cortex_agent")
workflow.add_edge("prefrontal_cortex_agent", "behavior_planner_agent")
workflow.add_edge("prefrontal_cortex_agent", "decision_maker_agent")
workflow.add_edge("prefrontal_cortex_agent", "social_behavior_modulator_agent")
workflow.add_edge("prefrontal_cortex_agent", "complex_thought_planner_agent")
workflow.add_edge("prefrontal_cortex_agent", "personality_expression_planner_agent")
workflow.add_edge(["behavior_planner_agent","decision_maker_agent", "social_behavior_modulator_agent", "complex_thought_planner_agent", "personality_expression_planner_agent"], "prefrontal_cortex_out_agent")
#workflow.add_edge("decision_maker_agent", "prefrontal_cortex_out_agent")
#workflow.add_edge("social_behavior_modulator_agent", "prefrontal_cortex_out_agent")
#workflow.add_edge("complex_thought_planner_agent", "prefrontal_cortex_out_agent")
#workflow.add_edge("personality_expression_planner_agent", "prefrontal_cortex_out_agent")
workflow.add_edge("prefrontal_cortex_out_agent", END)
#workflow.add_edge("generic_agent", END)

graph = workflow.compile()

In [7]:
from langchain_core.messages import HumanMessage

for s in graph.stream(
    {
        "input": [
            HumanMessage(content="Oi?")
        ]
    }
):
    if "__end__" not in s:
        print(s)
        print("----")


{'prefrontal_cortex_agent': {'behavior_planner': 'Desenvolver um plano para iniciar uma conversa, incluindo tópicos leves ou perguntas abertas para engajar o humano.', 'decision_maker': 'Avaliar o tom da interação e selecionar a resposta mais apropriada com base no contexto e na personalidade do agente.', 'social_behavior_modulator': 'Ajustar o nível de empatia e cordialidade na resposta, de acordo com as normas sociais de interação.', 'complex_thought_planner': 'Analisar possíveis direções para a conversa e preparar respostas para diferentes cenários que possam surgir a partir do cumprimento.', 'personality_expression_planner': 'Definir uma resposta que reflita uma personalidade amigável e acessível, garantindo que a interação seja agradável.'}}
----
{'social_behavior_modulator_agent': {'social_behavior': 'Acolhedor e amigável', 'guidelines': 'Manter um tom cordial e receptivo, utilizando uma linguagem simples e amigável.', 'examples_modulator': "Responder com um 'Oi! Como posso ajuda

## Invoke the team

With the graph created, we can now invoke it and see how it performs!


In [16]:
from langchain_core.messages import HumanMessage

for s in graph.stream(
    {
        "messages": [
            HumanMessage(content="Estou com alguns problemas, você pode me ajudar?")
        ]
    }
):
    if "__end__" not in s:
        print(s)
        print("----")

{'agent': {'messages': [AIMessage(content='Claro! Estou aqui para ajudar. Quais problemas você está enfrentando?', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 16, 'prompt_tokens': 207, 'total_tokens': 223, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_1bb46167f9', 'finish_reason': 'stop', 'logprobs': None}, id='run-3f3bfdb1-bfe1-4e67-b6f9-fa7b5cda9bf8-0', usage_metadata={'input_tokens': 207, 'output_tokens': 16, 'total_tokens': 223})]}}
----


In [ ]:
for s in graph.stream(
    {"messages": [HumanMessage(content="Estou com alguns problemas, você pode me ajudar?")]},
    {"recursion_limit": 5},
):
    if "__end__" not in s:
        print(s)
        print("----")

In [15]:
def print_stream(stream):
    for s in stream:
        message = s["messages"][-1]
        if isinstance(message, tuple):
            print(message)
        else:
            message.pretty_print()

inputs = {"messages": [("user", "Estou com alguns problemas, você pode me ajudar?")]}

print_stream(graph.stream(inputs, stream_mode="values"))

================================ Human Message =================================

Estou com alguns problemas, você pode me ajudar?
================================== Ai Message ==================================

Claro! Estou aqui para ajudar. Quais problemas você está enfrentando?
